## AOC 2024/03
### Part 1

--- Day 3: Mull It Over ---
"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 ) do nothing.

For example, consider the following section of corrupted memory:

```
xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
```

Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (2*4 + 5*5 + 11*8 + 8*5).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?

**AI Prompt**

I'm a skilled python developer, please help me consider various approaches for solving this problem and their pros/cons. I prefer simpler and understandable solutions, easily readable also for other developers. It's ok to generate code with testcases without explanation. I'll ask if I do not understand it. Always wrap test code into a function, to not clutter the global variable namespace.

**AI Response**

For this problem, I see two main approaches we could take:

1. Regular Expression (Regex) Approach:
- Pros: Clean, concise, handles pattern matching elegantly
- Cons: Regex can be harder to read/maintain for others
- Would look for pattern like `mul\((\d{1,3}),(\d{1,3})\)`

2. String Parsing Approach:
- Pros: More explicit, easier to understand the logic
- Cons: Might be more verbose
- Could use string methods like find() and split()

Which approach would you prefer to explore first? I can provide a test case implementation for either.

Also, would you like me to include some edge cases in the tests, like handling whitespace or invalid number formats?

In [ ]:
sample_input = """
xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
""".strip()

**AI Prompt**

use regex

**AI Response**

I'll create a test function using the regex approach. Here's an implementation:

```python
import re

def solve_mul(text: str) -> int:
    pattern = r'mul\((\d{1,3}),(\d{1,3})\)'
    matches = re.finditer(pattern, text)
    total = 0
    for match in matches:
        a, b = map(int, match.groups())
        total += a * b
    return total

def test_mul():
    assert solve_mul(sample_input) == 161
    assert solve_mul("mul(44,46)") == 2024
    assert solve_mul("mul(4*,5) mul(2,3) mul(6,9!") == 6  # only mul(2,3) is valid
    assert solve_mul("mul ( 2 , 4 )mul(3,4)") == 12  # spaces in first mul make it invalid
    print("All tests passed!")

test_mul()
```

Would you like me to explain any part of this implementation or add more test cases?

In [ ]:
import re

def solve_mul(text: str) -> int:
    pattern = r'mul\((\d{1,3}),(\d{1,3})\)'
    matches = re.finditer(pattern, text)
    total = 0
    for match in matches:
        a, b = map(int, match.groups())
        total += a * b
    return total

def test_mul():
    assert solve_mul(sample_input) == 161
    assert solve_mul("mul(44,46)") == 2024
    assert solve_mul("mul(4*,5) mul(2,3) mul(6,9!") == 6  # only mul(2,3) is valid
    assert solve_mul("mul ( 2 , 4 )mul(3,4)") == 12  # spaces in first mul make it invalid
    print("All tests passed!")

test_mul()

All tests passed!


**AI Prompt**

load full data

**AI Response**

Here's how we can load and process the actual puzzle input using the `advent-of-code-data` library:

```python
from aocd import get_data

def load_and_solve():
    data = get_data(day=3, year=2024)
    result = solve_mul(data)
    print(f"Solution: {result}")

load_and_solve()
```

Would you like me to explain anything about the data loading process or help troubleshoot if there are any issues?

In [ ]:
from aocd import get_data

full_input = get_data(day=3, year=2024)
print(len(full_input))

19264


In [ ]:
def solve_part1():
    return solve_mul(full_input)

solve_part1()

187194524

that was correct answer

### Part 2

--- Part Two ---
As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

- The do() instruction enables future mul instructions.
- The don't() instruction disables future mul instructions.
- Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

```
xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))
```

This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (2*4 + 8*5).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?

**AI Prompt**

adapt to updated problem

**AI Response**

For part 2, we'll need to track the enabled/disabled state as we process the string. Here's a proposed modification:

```python
def solve_mul_part2(text: str) -> int:
    total = 0
    enabled = True
    
    # Find all relevant instructions in order of appearance
    pattern = r'(?:do|don\'t)\(\)|mul\((\d{1,3}),(\d{1,3})\)'
    for match in re.finditer(pattern, text):
        if match.group(0) == 'do()':
            enabled = True
        elif match.group(0) == "don't()":
            enabled = False
        elif enabled:  # it's a mul instruction and enabled
            a, b = map(int, match.groups())
            total += a * b
    return total

def test_mul_part2():
    test_input = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)do()?mul(8,5))"
    assert solve_mul_part2(test_input) == 48
    print("Test passed!")

test_mul_part2()
```

Would you like me to explain the changes or shall we try it with the full input?

In [ ]:
def solve_mul_part2(text: str) -> int:
    total = 0
    enabled = True
    
    # Find all relevant instructions in order of appearance
    pattern = r'(?:do|don\'t)\(\)|mul\((\d{1,3}),(\d{1,3})\)'
    for match in re.finditer(pattern, text):
        if match.group(0) == 'do()':
            enabled = True
        elif match.group(0) == "don't()":
            enabled = False
        elif enabled:  # it's a mul instruction and enabled
            a, b = map(int, match.groups())
            total += a * b
    return total

def test_mul_part2():
    test_input = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)do()?mul(8,5))"
    assert solve_mul_part2(test_input) == 48
    print("Test passed!")

test_mul_part2()

Test passed!


In [ ]:
def solve_part2():
    return solve_mul_part2(full_input)

solve_part2()

127092535